In [1]:
import pandas as pd
import pickle
import pymongo

Δεδομένων όλων των ομιλιών, πρέπει να ανιχνεύσουμε ομοιότητες ανά ζεύγη μεταξύ των μελών του κοινοβουλίου. Συγκεκριμένα, πρέπει να βρούμε έναν τρόπο να εξαγάγουμε ένα διάνυσμα χαρακτηριστικών για κάθε μέλος και στη συνέχεια να εκτελέσουμε ομοιότητες ανά ζεύγη για να μπορέσουμε να ανιχνεύσουμε τα top-k ζεύγη με τον υψηλότερο βαθμό ομοιότητας (όπου k είναι μια παράμετρος).

In [2]:
# CONNECT TO DATABASE
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
client = mongo_client["GreekParliamentProceedings"]
index = client["InvertedIndex"]
database = client["Database"]

In [20]:
# READ KEYWORDS BY MEMBER
with open('../keywords/member_keywords.pickle', 'rb') as handle:
    keywords_by_member = pickle.load(handle)
keywords = keywords_by_member

### Διάνυσμα χαρακτηριστικών για κάθε μέλος

In [21]:
#CREATE A FEATURE VECTOR FOR EACH MEMBER
member_names = keywords_by_member.keys()
genders = []
political_parties = []
member_roles = []
regions = []
governments = []
def get_attributes_by_member(members, collection):
        """Creates an attribute vector for each member in a list of members.
        The attributes are gathered from a MongoDB collection.
        Each member has the following attributes: 
            name, gender, political parties, roles, regions, governments, keywords

        Parameters
        ----------
        members : list
            The names of the members
        
        collection: pymongo.collection.Collection
            A collection which includes information about the member attributes.
            The collection documents must include the following fields:
            'member_name', 'roles', 'member_gender', 'government', 'political_party', 'member_region'

        """
        
        genders = []
        political_parties = []
        member_roles = []
        regions = []
        governments = []
        for member in members:
                pipeline = [{'$match' : {'member_name':member}},
                            {'$facet': {
                                'groupByRoles':[{'$group':{'_id':'$roles'}}],
                                'groupByGender':[{'$group':{'_id':'$member_gender'}}],
                                'groupByGovernment':[{'$group':{'_id':'$government'}}],
                                'groupByParty':[{'$group':{'_id':'$political_party'}}],
                                'groupByRegion':[{'$group':{'_id':'$member_region'}}]
                            }
                                        }
                        ]
                r = list(collection.aggregate(pipeline))
                genders.append(r[0]['groupByGender'][0]['_id'])
                regions.append([r[0]['groupByRegion'][i]['_id'] for i in range(len(r[0]['groupByRegion']))])
                member_roles.append([r[0]['groupByRoles'][i]['_id'] for i in range(len(r[0]['groupByRoles']))])
                governments.append([r[0]['groupByGovernment'][i]['_id'] for i in range(len(r[0]['groupByGovernment']))])
                political_parties.append([r[0]['groupByParty'][i]['_id'] for i in range(len(r[0]['groupByParty']))])
                
        return genders, regions, political_parties, member_roles, governments

In [5]:
genders, regions, political_parties, member_roles, governments = get_attributes_by_member(member_names, database)

In [6]:
member_names = list(member_names)
print(member_names[1])

αβραμακης αντωνιου ελευθεριος


In [7]:
feature_vectors = []
member_names = list(member_names)
for i in range(len(keywords)):
    feature_vectors.append([])
    feature_vectors[i].append(member_names[i].split()[0])
    feature_vectors[i].append(genders[i])
    feature_vectors[i].extend(regions[i][j] for j in range(len(regions[i])))
    feature_vectors[i].extend(member_roles[i][j] for j in range(len(member_roles[i])))
    feature_vectors[i].extend(governments[i][j] for j in range(len(governments[i])))
    feature_vectors[i].extend(political_parties[i][j] for j in range(len(political_parties[i])))
    feature_vectors[i].extend(keywords[member_names[i]])

In [10]:
vector_dict = []
for i in range(len(keywords)):
    vector_dict.append({})
    vector_dict[i]['member_name'] = [member_names[i].split()[0]]
    vector_dict[i]['gender'] = [genders[i]]
    vector_dict[i]['member_roles'] = [member_roles[i][j] for j in range(len(member_roles[i]))]
    vector_dict[i]['governments'] = [governments[i][j] for j in range(len(governments[i]))]
    vector_dict[i]['political_parties'] = [political_parties[i][j] for j in range(len(political_parties[i]))]
    vector_dict[i]['keywords'] = keywords[member_names[i]]

In [12]:
print(vector_dict[1])

{'member_name': ['αβραμακης'], 'gender': ['male'], 'member_roles': ["['βουλευτης']"], 'governments': ["['μητσοτακη κυριακου(08/07/2019-28/07/2020)']"], 'political_parties': ['συνασπισμος ριζοσπαστικης αριστερας'], 'keywords': {'συριζ': 13, 'δημοκρατ': 10, 'νε': 7, 'διαταξ': 7, 'δικαιωμ': 7, 'σερρ': 6}}


In [13]:
#with open('feature_vectors.pickle', 'wb') as handle:
#   pickle.dump(feature_vectors, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [23]:
import pickle
with open('feature_vectors.pickle', 'rb') as handle:
    feature_vectors = pickle.load(handle)
with open('vector_dict.pickle', 'rb') as handle:
    vector_dict = pickle.load(handle)
from top_k_pairs import Top_k_Pairs as topk
top = topk()
pairs = top.get_pairs(10, feature_vectors, threshold=0.5)

In [24]:
keys = sorted(list(pairs))
print(keys)
k = 13
i = len(keys)-1
topk = {}
counter=0
if(k<=len(pairs.values())):
    while(counter<k):
        values = [ pairs[keys[i]][j] for j in range(len(pairs[keys[i]])) ]
        if(len(values)>=(k-counter)):
            topk[round(keys[i],3)]=values[:(k-counter)]
            break
        else:
            topk[round(keys[i],3)]=values
            counter = counter + len(values)
            i -= 1
        
print(topk)

[0.5135135135135135, 0.5142857142857142, 0.5151515151515151, 0.5172413793103449, 0.5185185185185185, 0.52, 0.5217391304347826, 0.5238095238095238, 0.5263157894736842, 0.5294117647058824, 0.5333333333333333, 0.5357142857142857, 0.5384615384615384, 0.5416666666666666, 0.5454545454545454, 0.5483870967741935, 0.55, 0.5526315789473685, 0.5555555555555556, 0.5652173913043478, 0.5714285714285714, 0.575, 0.5757575757575758, 0.5789473684210527, 0.5833333333333334, 0.5882352941176471, 0.5909090909090909, 0.6, 0.6111111111111112, 0.6176470588235294, 0.631578947368421, 0.6470588235294118, 0.7142857142857143, 0.7368421052631579]
{0.737: [[1120, 559]], 0.714: [[238, 1487]], 0.647: [[835, 967], [835, 204]], 0.632: [[624, 715], [871, 559]], 0.618: [[185, 1010]], 0.611: [[147, 414]], 0.6: [[1470, 374], [985, 204], [57, 439], [1120, 871]], 0.591: [[1240, 1076]]}


In [82]:
#for each similarity
for key in topk.keys():
    #for each pair in similarity
    for pair in topk[key]:
        #for each feature in the feature vector
        results = f"Members {member_names[pair[0]]} and {member_names[pair[1]]} have a jaccard similarity of {key}. They have similar: " + "\n"
        for feature in vector_dict[pair[0]].keys():
            intersection = set(vector_dict[pair[0]][feature]).intersection(set(vector_dict[pair[1]][feature]))
            #print(str(intersection))
            if(len(intersection)>0):
               results = results + " " + feature +": "+str(intersection) + "\n" 
    with open("topkpairs.txt", "a") as f:
        f.write(results)

In [117]:
print(len(({1,2}.intersection({3,4}))))

0
